# Playing with RSA

In [1]:
import timeit

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

## What happens if p and q are too close?

In [25]:
# p = next_prime(2^512)
p = random_prime(2^512-1, false, 2^511)

In [26]:
q=next_prime(p)

In [27]:
n_ = p * q

In [28]:
# n has 1024 bit
n_

135979990435737534809509083846264264753591599221670958233617927095019281078990888148663959427709779689624592418929358863145374314063765286226581212382936925718644072219862735591335602999999617081260103331445840777229782074453446469736660940106968419296341186247633656219400407013992646851783311468028524321249

In [29]:
# We start from the square root and look for the next prime
# next_prime(sqrt(n_).n(1000))
t = floor(sqrt(n_).n(1000))
t

11661045855142562417892522720178811202837980368318732343666226167794440012358778722705099782039549136646098726699208541443668254134946891928076908216764657

In [30]:
next_prime(t)

11661045855142562417892522720178811202837980368318732343666226167794440012358778722705099782039549136646098726699208541443668254134946891928076908216764737

In [31]:
# Compare with q
q

11661045855142562417892522720178811202837980368318732343666226167794440012358778722705099782039549136646098726699208541443668254134946891928076908216764737

## How long does it take to generate a key?

In [33]:
def gen():
    rsa.generate_private_key(
        public_exponent=int(65537),
        key_size=1024,
        backend=default_backend())
    
timeit.timeit(gen, number = 100)

1.9688302399999884

Homework: measure the key generation time for various key sizes up to 4096 bits. Plot a graph. 

# What happens if the exponent is too low?

In [39]:
private_key = rsa.generate_private_key(
    public_exponent=int(3),
    key_size=1024,
    backend=default_backend()
)
public_key = private_key.public_key()

In [40]:
message = b"OK"

m = int.from_bytes(message,"big")
c = power_mod(m,public_key.public_numbers().e,public_key.public_numbers().n)

print("m = {}, c = {}".format(m,c))

m = 20299, c = 8364190790899


In [41]:
# Can we find the message?
Integer(c).nth_root(3)

20299

In [45]:
# What happens if we use padding?
c = public_key.encrypt(message,
    padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None
    ))
print("c = {:d}".format(int.from_bytes(c,"big")))

c = 46906882650597165568434018910447187221941711572535893335974523606453645411049438586070914064302447077096600683108543613684561391593032741595654937577091545762999918697740855746072194841811460007402668668249887555366733229201927686995668960978856350652304769980888070317903874230757852929240478333451823575844
